In [ ]:
import pandapower as pp
import pandas as pd
import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt
%matplotlib inline

from pandapower.timeseries import DFData
from pandapower.timeseries import OutputWriter
from pandapower.timeseries.run_time_series import run_timeseries
from pandapower.control import ConstControl

In [ ]:
def NetworkBussesLines():
	net = pp.create_empty_network()

	b0 = pp.create_bus(net, 132, name='bus0', max_vm_pu=1.04, min_vm_pu=1.04)
	# The paper says 33 but it results in a trafo issue
	b1 = pp.create_bus(net, 20, name='bus1', max_vm_pu=1.1, min_vm_pu=0.9)
	b2 = pp.create_bus(net, 20, name='bus2', max_vm_pu=1.1, min_vm_pu=0.9)
	b3 = pp.create_bus(net, 20, name='bus3', max_vm_pu=1.1, min_vm_pu=0.9)
	b4 = pp.create_bus(net, 20, name='bus4', max_vm_pu=1.1, min_vm_pu=0.9)
	b5 = pp.create_bus(net, 20, name='bus5', max_vm_pu=1.1, min_vm_pu=0.9)

	pp.create_ext_grid(net, b0, max_p_mvar=200, max_q_mvar=200)
	trafo0 = pp.create_transformer(net, b0, b1, name="HV-MV transformer", std_type="25 MVA 110/20 kV")
	pp.create_line(net, b1, b2, 10, "149-AL1/24-ST1A 110.0", name='line12')
	pp.create_line(net, b1, b3, 5, "149-AL1/24-ST1A 110.0", name='line13')
	pp.create_line(net, b2, b4, 5, "149-AL1/24-ST1A 110.0", name='line24')
	pp.create_line(net, b2, b5, 5, "149-AL1/24-ST1A 110.0", name='line25')
	
	#Following paper's parameters for the line
	#Still c_nf_per_km (line capacitance in nano Farad per km, permittivity) and max_i_ka (maximum thermal current in kilo Ampere) missing
	# pp.create_line_from_parameters(net, b1, b2, 1, 0.03  , 0.022 , name='line12')
	# pp.create_line_from_parameters(net, b1, b3, 1, 0.0307, 0.0621, name='line13')
	# pp.create_line_from_parameters(net, b2, b4, 1, 0.0303, 0.0611, name='line24')
	# pp.create_line_from_parameters(net, b2, b5, 1, 0.0159, 0.0502, name='line25')

	return net, b0,b1,b2,b3,b4,b5

In [ ]:
#Case 1
def ANM_Network_case1():
	net,b0,b1,b2,b3,b4,b5 = NetworkBussesLines()

	#3rd-bus elements
	pp.create_load(net, b3, p_mw=1, q_mvar=-.2, max_p_mw=10, min_p_mw=0, max_q_mvar=2, min_q_mvar=-2, name='Load0') #, max_q_mvar=2 
	pp.create_sgen(net, b3, p_mw=0, q_mvar=0, max_p_mw=30, min_p_mw=0, max_q_mvar=30, name="PV generator")

	#4th-bus elements
	pp.create_load(net, b4, p_mw=4, q_mvar=-.8, max_p_mw=30, min_p_mw=0, max_q_mvar=6, name='Load1')
	pp.create_sgen(net, b4, p_mw=40, q_mvar=0, max_p_mw=50, min_p_mw=0, max_q_mvar=50, min_q_mvar=-50, name="Wind generator")

	#5th-bus elements
	pp.create_load(net, b5, p_mw=0, q_mvar=0, max_p_mw=30, min_p_mw=0, max_q_mvar=6, name='Load2')
	pp.create_storage(net, b5, p_mw=0, max_e_mwh=0.66, q_mvar=0, max_p_mw=50, min_p_mw=-50, max_q_mvar=50, min_q_mvar=-50, name="Battery unit")

	return net

In [ ]:
#Case 2
def ANM_Network_case2():
	net,b0,b1,b2,b3,b4,b5 = NetworkBussesLines()

	#3rd-bus elements
	pp.create_load(net, b3, p_mw=5, q_mvar=-1, max_p_mw=10, min_p_mw=0, max_q_mvar=2, min_q_mvar=-2)
	pp.create_sgen(net, b3, p_mw=4, q_mvar=0, max_p_mw=30, min_p_mw=0, max_q_mvar=30, min_q_mvar=-30, name="PV generator")

	#4th-bus elements
	pp.create_load(net, b4, p_mw=10, q_mvar=-2, max_p_mw=30, min_p_mw=0, max_q_mvar=6)
	pp.create_sgen(net, b4, p_mw=11, q_mvar=0, max_p_mw=50, min_p_mw=0, max_q_mvar=50, min_q_mvar=-50, name="Wind generator")

	#5th-bus elements
	pp.create_load(net, b5, p_mw=25, q_mvar=-5, max_p_mw=30, min_p_mw=0, max_q_mvar=6)
	pp.create_storage(net, b5, p_mw=0, max_e_mwh=0.66, q_mvar=0, max_p_mw=50, min_p_mw=-50, max_q_mvar=50, min_q_mvar=-50, name="Battery unit")

	return net

In [ ]:
#Case 3
def ANM_Network_case3():
	net,b0,b1,b2,b3,b4,b5 = NetworkBussesLines()

	#3rd-bus elements
	pp.create_load(net, b3, p_mw=2, q_mvar=-.4, max_p_mw=10, min_p_mw=0, max_q_mvar=2, min_q_mvar=-2) #, min_q_mvar= 
	pp.create_sgen(net, b3, p_mw=30, q_mvar=0, max_p_mw=30, min_p_mw=0, max_q_mvar=30, min_q_mvar=-30, name="PV generator")

	#4th-bus elements
	pp.create_load(net, b4, p_mw=20, q_mvar=-4, max_p_mw=30, min_p_mw=0, max_q_mvar=6)
	pp.create_sgen(net, b4, p_mw=40, q_mvar=0, max_p_mw=50, min_p_mw=0, max_q_mvar=50, min_q_mvar=-50, name="Wind generator")

	#5th-bus elements
	pp.create_load(net, b5, p_mw=0, q_mvar=0, max_p_mw=30, min_p_mw=0, max_q_mvar=6)
	pp.create_storage(net, b5, p_mw=0, max_e_mwh=0.75, q_mvar=0, max_p_mw=50, min_p_mw=-50, max_q_mvar=50, min_q_mvar=-50, name="Battery unit")

	return net

In [ ]:
def ANM_Network_cust():
	net,b0,b1,b2,b3,b4,b5 = NetworkBussesLines()

	#3rd-bus elements
	#max p_mw = 15
	pp.create_load(net, b3, p_mw=0, q_mvar=0, 
						max_p_mw=10, min_p_mw=0, max_q_mvar=2, min_q_mvar=-2) #, min_q_mvar= 
	#max p_mw = 30
	pp.create_sgen(net, b3, p_mw=0, q_mvar=0, 
						max_p_mw=30, min_p_mw=0, max_q_mvar=30, min_q_mvar=-30, name="PV generator")

	#4th-bus elements
	pp.create_load(net, b4, p_mw=0, q_mvar=0, 
						max_p_mw=30, min_p_mw=0, max_q_mvar=6)
	# p_mw=44
	pp.create_sgen(net, b4, p_mw=0*0, q_mvar=0, 
						max_p_mw=50, min_p_mw=0, max_q_mvar=50, min_q_mvar=-50, name="Wind generator")

	#5th-bus elements
	pp.create_load(net, b5, p_mw=0, q_mvar=0, 
						max_p_mw=30, min_p_mw=0, max_q_mvar=6)
	pp.create_storage(net, b5, p_mw=10, max_e_mwh=0.75, q_mvar=0, max_p_mw=50, min_p_mw=-50, max_q_mvar=50, min_q_mvar=-50, name="Battery unit")

	return net

In [ ]:
case = 1
if(case==1):
	net = ANM_Network_case1()
elif(case==2):
	net = ANM_Network_case2()
elif(case==3):
	net = ANM_Network_case3()

In [ ]:
#Check for network issues
out = pp.diagnostic(net)

In [ ]:
#Normal run
pp.runpp(net)

In [ ]:
#Multi tests
# for i in range(0,16,1):
# 	net = ANM_Network_cust(i)
# 	pp.runpp(net)
# 	print(f'Val: {i}')
# 	fig = pf_res_plotly(net)

Time series run

In [ ]:
np.cos(range(n_timesteps))

In [ ]:
#Generate Data
output_dir = './time_series_example'
n_timesteps = 4 * 24 * 365

profile = pd.DataFrame()
loads_names = ['load0_p', 'load1_p', 'load2_p']
profile[loads_names[0]] = np.cos(range(n_timesteps))**2 * 10
profile[loads_names[1]] = np.random.random(n_timesteps) * np.random.normal(1,.5) * np.random.random(n_timesteps) * 9
profile[loads_names[2]] = np.random.normal(2,0.5) * np.random.normal(4,4.5)
profile_load = profile
ds = DFData(profile)
ConstControl(net, 'load', 'p_mw', element_index=net.load.index, profile_name=profile.columns, data_source=ds)

profile = pd.DataFrame()
generators_names = ['PV gen_p', 'Wind gen_p']
profile[generators_names[0]] = np.random.random(n_timesteps) * np.random.random(n_timesteps)**3 * 3
profile[generators_names[1]] = np.cos(range(n_timesteps))**2 * 12
profile_sgen = profile
ds = DFData(profile)
ConstControl(net, 'sgen', 'p_mw', element_index=net.sgen.index, profile_name=profile.columns, data_source=ds)

#Test the max and min values
#Max
print('Check max values')
max_value = 16
max_loads = profile_load.max()
max_sgens = profile_sgen.max()
for ml in max_loads:
	if ml>max_value:
		print(f"Possible overflow, load too large (val>{max_value})")
		print(max_loads)
for ms in max_sgens:
	if ms>max_value:
		print(f"Possible overflow, gen too large (val>{max_value})")
		print(max_sgens)
#Min
print('Check min values')
min_loads = profile_load.min()
min_sgens = profile_sgen.min()
for ml in min_loads:
	if ml<0:
		print("Negative load. It should be positive")
		print(min_loads)
for ms in min_sgens:
	if ms<0:
		print("Negative gen. It should be positive")
		print(min_sgens)
# print(max_loads)
# print(max_sgens)
# print(min_loads)
# print(min_sgens)

In [ ]:
time_steps = range(0,n_timesteps)

ow = OutputWriter(net, time_steps, output_path=output_dir, output_file_type=".xlsx", log_variables=list())
# these variables are saved to the harddisk after / during the time series loop
ow.log_variable('res_load', 'p_mw')
ow.log_variable('res_bus', 'vm_pu')
ow.log_variable('res_line', 'loading_percent')
ow.log_variable('res_line', 'i_ka')

run_timeseries(net,time_steps)

path = os.path.join(output_dir, "loads_p.xlsx")
profile_load.to_excel(path)
path = os.path.join(output_dir, "sgen_p.xlsx")
profile_sgen.to_excel(path)

In [ ]:
#Plot network and loadings
from pandapower.plotting.plotly import simple_plotly
from pandapower.plotting.plotly import vlevel_plotly
from pandapower.plotting.plotly import pf_res_plotly

# _ = simple_plotly(net)
# _ = vlevel_plotly(net)
fig = pf_res_plotly(net)
fig.write_html(f"images/fig_case{case}.html")

In [ ]:
#Plot time series
import os

# voltage results
vm_pu_file = os.path.join(output_dir, "res_bus", "vm_pu.xlsx")
vm_pu = pd.read_excel(vm_pu_file, index_col=0)
vm_pu.iloc[:,1:].plot(label="vm_pu")
plt.xlabel("time step")
plt.ylabel("voltage mag. [p.u.]")
plt.title("Voltage Magnitude (busses)")
plt.grid()
plt.show()

# line loading results
ll_file = os.path.join(output_dir, "res_line", "loading_percent.xlsx")
line_loading = pd.read_excel(ll_file, index_col=0)
line_loading.plot(label="line_loading")
plt.xlabel("time step")
plt.ylabel("line loading [%]")
plt.title("Line Loading")
plt.grid()
plt.show()

# load results
load_file = os.path.join(output_dir, "res_load", "p_mw.xlsx")
load = pd.read_excel(load_file, index_col=0)
load.plot(label="load")
plt.xlabel("time step")
plt.ylabel("P [MW]")
plt.title("Loads")
plt.grid()
plt.show()

In [ ]:
#Generate timeseries Dataset
output_dir = './time_series_example'
load_p_file = os.path.join(output_dir, "loads_p.xlsx")
load_p = pd.read_excel(load_p_file, index_col=0)
sgen_p_file = os.path.join(output_dir, "sgen_p.xlsx")
sgen_p_loading = pd.read_excel(sgen_p_file, index_col=0)
x = pd.concat([load_p, sgen_p_loading], axis=1)

ll_file = os.path.join(output_dir, "res_line", "loading_percent.xlsx")
y = pd.read_excel(ll_file, index_col=0)


# vm_pu_file = os.path.join(output_dir, "res_bus", "vm_pu.xlsx")
# y = pd.read_excel(vm_pu_file, index_col=0)
# y = y.iloc[:,1:]

In [ ]:
y = y.rename(columns={0: "L0", 1: "L1", 2: "L2", 3: "L3"})
y

In [ ]:
df = pd.concat([x,y], axis=1)

n = len(df)
train_df = df[0:int(n*0.7)]
val_df = df[int(n*0.7):int(n*0.9)]
test_df = df[int(n*0.9):]

train_mean = train_df.mean()
train_std = train_df.std()

train_df = (train_df - train_mean) / train_std
val_df = (val_df - train_mean) / train_std
test_df = (test_df - train_mean) / train_std

print(train_df.shape,val_df.shape,test_df.shape)

In [ ]:
(test_df * train_std + train_mean)[train_df.columns[-4:]].plot()

In [ ]:
class WindowGenerator():
	def __init__(self, input_width, label_width, shift,
				train_df=train_df, val_df=val_df, test_df=test_df,
				all_columns =None, training_columns=None, label_columns=None):
		# Store the raw data.
		self.train_df = train_df
		self.val_df = val_df
		self.test_df = test_df

		self.all_columns = all_columns
		self.column_indices = {name: i for i, name in enumerate(self.all_columns)}
		print('All columns: ', self.column_indices)

		self.training_columns = training_columns
		self.column_indices_t = {name: i for i, name in enumerate(self.training_columns)}
		print('Train columns: ', self.column_indices_t)

		self.label_columns = label_columns
		self.column_indices_l = {name: i for i, name in enumerate(label_columns)}
		print('Label columns: ', self.column_indices_l)

		# Work out the window parameters.
		self.input_width = input_width
		self.label_width = label_width
		self.shift = shift

		self.total_window_size = input_width + shift

		self.input_slice = slice(0, input_width)
		self.input_indices = np.arange(self.total_window_size)[self.input_slice]

		self.label_start = self.total_window_size - self.label_width
		self.labels_slice = slice(self.label_start, None)
		self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

	def __repr__(self):
		return '\n'.join([
			f'Total window size: {self.total_window_size}',
			f'Input indices: {self.input_indices}',
			f'Label indices: {self.label_indices}',
			f'Label column name(s): {self.label_columns}'])

	def split_window(self, features):
		inputs = features[:, self.input_slice, :]
		a = [self.all_columns.index(name) for name in self.training_columns]
		print(f'Train cols: {a}, \n{self.training_columns}, \n{self.column_indices}')
		# inputs = tf.stack( [inputs[:, :, self.column_indices_t[name]] for name in self.training_columns], axis=-1)
		inputs = tf.stack( [inputs[:, :, self.all_columns.index(name)] for name in self.training_columns], axis=-1)

		labels = features[:, self.labels_slice, :]
		if self.label_columns is not None:
			a = [self.all_columns.index(name) for name in self.label_columns]
			print(f'Label cols: {a}, \n{self.label_columns}, \n{self.column_indices}')
			labels = tf.stack( [labels[:, :, self.all_columns.index(name)] for name in self.label_columns], axis=-1)

		# Slicing doesn't preserve static shape information, so set the shapes
		# manually. This way the `tf.data.Datasets` are easier to inspect.
		inputs.set_shape([None, self.input_width, None])
		labels.set_shape([None, self.label_width, None])

		return inputs, labels

	def plot(self, model=None):
		inputs, labels = self.example
		print(inputs.shape, labels.shape)
		t = len(self.all_columns)
		l = len(self.label_columns)
		plt.figure(figsize=(20, 6*l))

		batch_index = 0
		for i,c in enumerate(self.label_columns):
			plt.subplot(l, 1, i+1)
			plt.ylabel(f'{c}')
			plt.plot(self.input_indices, inputs[batch_index, :, (t-l)+i], label='Inputs', marker='.', zorder=-10) #inputs

			plt.scatter(self.label_indices, labels[batch_index, :, i],
						edgecolors='k', label='Labels', c='#2ca02c', s=64)

			if model is not None:
				predictions = model(inputs)
				plt.scatter(self.label_indices, predictions[batch_index, :, i],
							marker='X', edgecolors='k', label='Predictions',
							c='#ff7f0e', s=64)

			if i == 0:
				plt.legend()

		plt.xlabel('Time [...]')

	def make_dataset(self, data):
		data = np.array(data, dtype=np.float32)
		ds = tf.keras.utils.timeseries_dataset_from_array(
			data=data,
			targets=None,
			sequence_length=self.total_window_size,
			sequence_stride=1,
			shuffle=True,
			batch_size=32,)

		ds = ds.map(self.split_window)

		return ds

	@property
	def train(self):
		return self.make_dataset(self.train_df)

	@property
	def val(self):
		return self.make_dataset(self.val_df)

	@property
	def test(self):
		return self.make_dataset(self.test_df)

	@property
	def example(self):
		"""Get and cache an example batch of `inputs, labels` for plotting."""
		result = getattr(self, '_example', None)
		if result is None:
			# No example batch was found, so get one from the `.train` dataset
			result = next(iter(self.test))
			# And cache it for next time
			self._example = result
		return result

In [ ]:
all_columns = train_df.columns
columns_training =  all_columns #y.columns #train_df.columns
columns_label = y.columns
input_window = 12
output_window = 6
shift = output_window
dataset_windows = WindowGenerator(
	input_width=input_window, label_width=output_window, shift=shift, train_df=train_df, val_df=val_df, test_df=test_df, all_columns=list(all_columns), training_columns=list(columns_training), label_columns=list(columns_label))


In [ ]:
for example_inputs, example_labels in dataset_windows.train.take(1):
  print(f'Inputs shape (batch, time, features): {example_inputs.shape}')
  print(f'Labels shape (batch, time, features): {example_labels.shape}')


In [ ]:
class CustomLoss(tf.keras.metrics.Metric):
	def __init__(self, n_columns, name='mae_custom', **kwargs):
		super().__init__(name, **kwargs)
		#initialiaze the variables used to calculate the loss
		self.count = self.add_weight(name='count', initializer='zeros')
		#The shape [2]('shape=(2,)' is equivalent) is for temperature ad humidity
		self.total = self.add_weight(name='total', initializer='zeros', shape=(n_columns,))

	#Called at every batch of data
	def update_state(self, y_true, y_pred, sample_weight=None):
		# print('Prediction ',y_pred)
		# print('True ',y_true.numpy())
		error = tf.abs(y_pred-y_true)
		error = tf.reduce_mean(error, axis=(0,1))#
		# print('Error: ', error,'\n')
		#You can just use + sign but it is better to use assign_add method
		self.total.assign_add(error)
		self.count.assign_add(1.)
		return
	def reset_state(self):
		self.count.assign(tf.zeros_like(self.count))
		self.total.assign(tf.zeros_like(self.total))
		return
	def result(self):
		results = tf.math.divide_no_nan(self.total, self.count)
		return results



In [ ]:
dense = tf.keras.Sequential([
	tf.keras.layers.Flatten(),
	tf.keras.layers.Dense(units=64*2, activation='relu'),
	tf.keras.layers.Dense(units=64, activation='relu'),
	tf.keras.layers.Dense(units=output_window*len(columns_label)),
	tf.keras.layers.Reshape([output_window, len(columns_label)])
])
cnn = tf.keras.Sequential([
		tf.keras.layers.Conv1D(filters=int(64),kernel_size=(3,),activation='relu'),
		tf.keras.layers.Flatten(),
		tf.keras.layers.Dense(int(64), activation='relu'),
		tf.keras.layers.Dense(units=output_window*len(columns_label)),
		tf.keras.layers.Reshape([output_window, len(columns_label)])
	])
rnn = tf.keras.Sequential([
		tf.keras.layers.LSTM(units=int(64)),
		tf.keras.layers.Flatten(),
		tf.keras.layers.Dense(units=output_window*len(columns_label)),
		tf.keras.layers.Reshape([output_window, len(columns_label)])
	])


In [ ]:
MAX_EPOCHS = 10

monitor = 'val_mean_absolute_error'
def compile_and_fit(model, window, patience=5):
	early_stopping = tf.keras.callbacks.EarlyStopping(monitor=monitor,
													patience=patience,
													mode='min',
													verbose=1)

	model.compile(loss=tf.losses.MeanSquaredError(),
				optimizer=tf.optimizers.Adam(),
				metrics=[tf.metrics.MeanAbsoluteError(),CustomLoss(len(columns_label))])

	history = model.fit(window.train, epochs=MAX_EPOCHS,
						validation_data=window.val,
						callbacks=[early_stopping])
	return history

In [ ]:
# tf.config.run_functions_eagerly(True) #to use tf_ds.numpy()
history = compile_and_fit(dense, dataset_windows)

In [ ]:
loss_plot = 'mean_absolute_error'
plt.plot(history.epoch, history.history[loss_plot], label=loss_plot)
plt.plot(history.epoch, history.history[monitor], label=monitor)
plt.legend()

In [ ]:
dataset_windows.plot(dense)

In [ ]:
error = dense.evaluate(single_step_window.test, verbose=1)
print(error)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x,y,train_size=.8)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

scaler = StandardScaler()
y_train = np.array(scaler.fit_transform(y_train))
y_test = np.array(y_test)

In [ ]:
from sklearn.neural_network import MLPRegressor

ann = MLPRegressor(verbose=1)
ann.fit(X_train,y_train)

y_prediction = ann.predict(X_test)
y_prediction = scaler.inverse_transform(y_prediction)

In [ ]:
start = 140
end = start * 3
col = 1
fig = plt.figure(figsize=(20,10))
plt.plot(y_test[start:end,col], alpha=0.5, linestyle='--', label='correct values')
plt.plot(y_prediction[start:end,col], alpha=0.5, linestyle='-', label='predicted values')
plt.legend()
plt.show()


In [ ]:

from sklearn.metrics import r2_score
for i in range(0,y_test.shape[-1]):
	r2_score_val = r2_score(y_test[:,i], y_prediction[:,i])
	print(f'{i}, The R2 score is: {r2_score_val:.3f}')# ({(r2_score*100):.1f}%)